In [1]:
# ALWAYS RUN THIS FIRST!
import os
import sys
from pathlib import Path

NOTEBOOK_DIR = Path("/rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest")
os.chdir(NOTEBOOK_DIR)
sys.path.insert(0, str(NOTEBOOK_DIR))

print(f"✅ Working directory: {os.getcwd()}")


✅ Working directory: /rsrch9/home/plm/idso_fa1_pathology/codes/yshokrollahi/vitamin-p-latest


In [5]:
import pandas as pd
import json
from shapely.geometry import Polygon, mapping
from collections import defaultdict
import numpy as np
from tqdm import tqdm

print("="*60)
print("CONVERTING XENIUM CELL BOUNDARIES TO GEOJSON (FULL)")
print("="*60)

# Read Xenium cell boundaries
data_dir = "/rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/Lung_Anthracosis/output-XETG00522__0057986__Region_1__20251203__234028"
boundaries_path = f"{data_dir}/cell_boundaries.parquet"

print(f"\nReading: {boundaries_path}")
boundaries_df = pd.read_parquet(boundaries_path)

print(f"✅ Loaded {len(boundaries_df):,} vertices")
print(f"   Unique cells: {boundaries_df['cell_id'].nunique():,}")

# Xenium pixel size
pixel_size = 0.2125  # microns per pixel

print(f"\nCoordinate conversion:")
print(f"  Input: MICRONS")
print(f"  Output: PIXELS")
print(f"  Pixel size: {pixel_size} μm/pixel")

# Convert microns to pixels
print("\nConverting coordinates...")
boundaries_df['vertex_x_pixels'] = boundaries_df['vertex_x'] / pixel_size
boundaries_df['vertex_y_pixels'] = boundaries_df['vertex_y'] / pixel_size

print(f"✅ Converted coordinate ranges:")
print(f"  X (pixels): {boundaries_df['vertex_x_pixels'].min():.1f} - {boundaries_df['vertex_x_pixels'].max():.1f}")
print(f"  Y (pixels): {boundaries_df['vertex_y_pixels'].min():.1f} - {boundaries_df['vertex_y_pixels'].max():.1f}")

# Group vertices by cell_id
print("\nGrouping vertices by cell...")
cells = defaultdict(list)
for row in tqdm(boundaries_df.itertuples(), total=len(boundaries_df), desc="Processing vertices"):
    cells[row.cell_id].append([row.vertex_x_pixels, row.vertex_y_pixels])

print(f"✅ Grouped into {len(cells):,} cells")

# Create GeoJSON
print("\nCreating GeoJSON polygons...")
features = []
skipped = 0

for cell_id, vertices in tqdm(cells.items(), desc="Creating polygons"):
    # Make sure polygon is closed
    if vertices[0] != vertices[-1]:
        vertices.append(vertices[0])
    
    try:
        polygon = Polygon(vertices)
        
        if polygon.is_valid and polygon.area > 0:
            # Calculate area in both units
            area_pixels = polygon.area
            area_um2 = area_pixels * (pixel_size ** 2)
            
            feature = {
                "type": "Feature",
                "geometry": mapping(polygon),
                "properties": {
                    "cell_id": str(cell_id),
                    "area_pixels": float(area_pixels),
                    "area_um2": float(area_um2),
                    "source": "xenium"
                }
            }
            features.append(feature)
        else:
            skipped += 1
    except Exception as e:
        skipped += 1
        continue

print(f"✅ Created {len(features):,} valid polygons")
if skipped > 0:
    print(f"   ⚠️  Skipped {skipped:,} invalid cells")

# Create GeoJSON FeatureCollection
geojson = {
    "type": "FeatureCollection",
    "features": features
}

# Save to results_new directory
output_path = "results_new/xenium_cell_boundaries_FULL.geojson"
print(f"\nSaving to: {output_path}")
print(f"   (This may take a moment for large files...)")

with open(output_path, 'w') as f:
    json.dump(geojson, f)

file_size_mb = os.path.getsize(output_path) / (1024 * 1024)
print(f"✅ Saved Xenium boundaries as GeoJSON ({file_size_mb:.1f} MB)")

# Print statistics
areas_um2 = [f['properties']['area_um2'] for f in features]
areas_pixels = [f['properties']['area_pixels'] for f in features]

print(f"\nXenium cell statistics (FULL dataset):")
print(f"  Total cells: {len(features):,}")
print(f"  Mean area: {np.mean(areas_um2):.2f} μm² ({np.mean(areas_pixels):.1f} pixels²)")
print(f"  Median area: {np.median(areas_um2):.2f} μm² ({np.median(areas_pixels):.1f} pixels²)")
print(f"  Min area: {np.min(areas_um2):.2f} μm²")
print(f"  Max area: {np.max(areas_um2):.2f} μm²")
print(f"  Std area: {np.std(areas_um2):.2f} μm²")

print(f"\n{'='*60}")
print("FULL CONVERSION COMPLETE!")
print(f"{'='*60}")
print(f"Output: {output_path}")
print(f"File size: {file_size_mb:.1f} MB")
print(f"\nNow you can compare:")
print(f"  • Xenium: results_new/xenium_cell_boundaries_FULL.geojson")
print(f"  • Vitamin-P: results_new/instances.geojson")
print(f"\n⏱️  Estimated processing time: ~5-10 minutes")

CONVERTING XENIUM CELL BOUNDARIES TO GEOJSON (FULL)

Reading: /rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/Lung_Anthracosis/output-XETG00522__0057986__Region_1__20251203__234028/cell_boundaries.parquet
✅ Loaded 27,769,869 vertices
   Unique cells: 1,111,495

Coordinate conversion:
  Input: MICRONS
  Output: PIXELS
  Pixel size: 0.2125 μm/pixel

Converting coordinates...
✅ Converted coordinate ranges:
  X (pixels): 96.0 - 54200.0
  Y (pixels): 72.0 - 112123.0

Grouping vertices by cell...


Processing vertices: 100%|██████████| 27769869/27769869 [00:37<00:00, 740525.02it/s] 


✅ Grouped into 1,111,495 cells

Creating GeoJSON polygons...


Creating polygons: 100%|██████████| 1111495/1111495 [01:57<00:00, 9444.60it/s] 


✅ Created 1,110,339 valid polygons
   ⚠️  Skipped 1,156 invalid cells

Saving to: results_new/xenium_cell_boundaries_FULL.geojson
   (This may take a moment for large files...)
✅ Saved Xenium boundaries as GeoJSON (1185.0 MB)

Xenium cell statistics (FULL dataset):
  Total cells: 1,110,339
  Mean area: 75.95 μm² (1682.0 pixels²)
  Median area: 60.74 μm² (1345.0 pixels²)
  Min area: 1.42 μm²
  Max area: 984.99 μm²
  Std area: 52.83 μm²

FULL CONVERSION COMPLETE!
Output: results_new/xenium_cell_boundaries_FULL.geojson
File size: 1185.0 MB

Now you can compare:
  • Xenium: results_new/xenium_cell_boundaries_FULL.geojson
  • Vitamin-P: results_new/instances.geojson

⏱️  Estimated processing time: ~5-10 minutes


## Geojson to parquet

In [ ]:
import pandas as pd
import json
import numpy as np
from tqdm import tqdm

print("="*60)
print("CONVERTING VITAMIN-P RESULTS TO PARQUET FORMAT")
print("="*60)

# Read Vitamin-P GeoJSON results
vitaminp_geojson_path = "results_new/instances.geojson"

print(f"\nReading Vitamin-P results: {vitaminp_geojson_path}")
with open(vitaminp_geojson_path, 'r') as f:
    geojson_data = json.load(f)

num_cells = len(geojson_data['features'])
print(f"✅ Loaded {num_cells:,} cells from GeoJSON")

# Xenium pixel size for converting back to microns
pixel_size = 0.2125  # microns per pixel

print(f"\nCoordinate conversion:")
print(f"  Input: PIXELS (from GeoJSON)")
print(f"  Output: MICRONS (Xenium format)")
print(f"  Pixel size: {pixel_size} μm/pixel")

# Convert to Parquet format (matching Xenium structure)
print("\nConverting to Parquet format...")
boundary_rows = []

for feature in tqdm(geojson_data['features'], desc="Processing cells"):
    cell_id = feature['properties'].get('cell_id', feature['properties'].get('id', 'unknown'))
    
    # Get polygon coordinates
    geometry = feature['geometry']
    if geometry['type'] == 'Polygon':
        # Polygon coordinates are [[[x1, y1], [x2, y2], ...]]
        coordinates = geometry['coordinates'][0]  # Exterior ring
    else:
        print(f"  ⚠️  Skipping non-polygon geometry: {geometry['type']}")
        continue
    
    # Convert each vertex from pixels to microns
    for vertex in coordinates:
        x_pixels, y_pixels = vertex
        x_microns = x_pixels * pixel_size
        y_microns = y_pixels * pixel_size
        
        boundary_rows.append({
            'cell_id': cell_id,
            'vertex_x': x_microns,
            'vertex_y': y_microns
        })

# Create DataFrame
print(f"\n✅ Converted {len(boundary_rows):,} vertices")
vitaminp_boundaries = pd.DataFrame(boundary_rows)

print(f"\nDataFrame info:")
print(f"  Unique cells: {vitaminp_boundaries['cell_id'].nunique():,}")
print(f"  Total vertices: {len(vitaminp_boundaries):,}")
print(f"  Columns: {list(vitaminp_boundaries.columns)}")

print(f"\nCoordinate ranges (microns):")
print(f"  vertex_x: {vitaminp_boundaries['vertex_x'].min():.2f} - {vitaminp_boundaries['vertex_x'].max():.2f}")
print(f"  vertex_y: {vitaminp_boundaries['vertex_y'].min():.2f} - {vitaminp_boundaries['vertex_y'].max():.2f}")

# Save as Parquet
output_path = "results_new/vitaminp_cell_boundaries.parquet"
print(f"\nSaving to: {output_path}")
vitaminp_boundaries.to_parquet(output_path, index=False)

file_size_mb = pd.io.parquet.read_parquet(output_path).memory_usage(deep=True).sum() / (1024**2)
print(f"✅ Saved as Parquet")

# Create cells metadata (centroids and areas)
print("\nCreating cells metadata...")
cells_metadata = []

for feature in tqdm(geojson_data['features'], desc="Creating metadata"):
    props = feature['properties']
    cell_id = props.get('cell_id', props.get('id', 'unknown'))
    
    # Get coordinates
    coordinates = feature['geometry']['coordinates'][0]
    
    # Calculate centroid
    x_coords = [v[0] for v in coordinates]
    y_coords = [v[1] for v in coordinates]
    centroid_x_pixels = np.mean(x_coords)
    centroid_y_pixels = np.mean(y_coords)
    
    # Convert to microns
    centroid_x_microns = centroid_x_pixels * pixel_size
    centroid_y_microns = centroid_y_pixels * pixel_size
    
    # Get area
    area_um2 = props.get('area_um', props.get('area_um2', 0))
    area_pixels = props.get('area_pixels', area_um2 / (pixel_size ** 2))
    
    cells_metadata.append({
        'cell_id': cell_id,
        'x_centroid': centroid_x_microns,
        'y_centroid': centroid_y_microns,
        'cell_area': area_um2,
        'area_pixels': area_pixels,
        'segmentation_method': 'vitamin_p'
    })

cells_df = pd.DataFrame(cells_metadata)
cells_output = "results_new/vitaminp_cells_metadata.parquet"
cells_df.to_parquet(cells_output, index=False)

print(f"✅ Saved cells metadata: {cells_output}")

# Print statistics
print(f"\n{'='*60}")
print("CONVERSION COMPLETE!")
print(f"{'='*60}")
print(f"\nOutput files:")
print(f"  1. Cell boundaries: {output_path}")
print(f"     • {vitaminp_boundaries['cell_id'].nunique():,} cells")
print(f"     • {len(vitaminp_boundaries):,} vertices")
print(f"  2. Cell metadata: {cells_output}")
print(f"     • Centroids in microns")
print(f"     • Cell areas in μm²")

print(f"\nCell statistics:")
print(f"  Mean area: {cells_df['cell_area'].mean():.2f} μm²")
print(f"  Median area: {cells_df['cell_area'].median():.2f} μm²")
print(f"  Min area: {cells_df['cell_area'].min():.2f} μm²")
print(f"  Max area: {cells_df['cell_area'].max():.2f} μm²")

print(f"\nFormat comparison:")
print(f"  Xenium format: cell_boundaries.parquet (microns)")
print(f"  Vitamin-P format: vitaminp_cell_boundaries.parquet (microns)")
print(f"  ✅ Both now in same coordinate system!")

## Figure 3

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import tifffile
import json
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION
# ============================================================================
data_dir = "/rsrch9/home/plm/idso_fa1_pathology/TIER1/paul-xenium/Lung_Anthracosis/output-XETG00522__0057986__Region_1__20251203__234028"
results_dir = "results_new"

# Input files
vitamin_p_geojson = f"{results_dir}/vitamin-p-cell_segmentation_10k.geojson"
xenium_geojson = f"{results_dir}/xenium_cell_boundaries_10k_crop.geojson"
morphology_image = "test_images/combined_2channel_crop_10000x10000.ome.tif"

# ROI parameters
roi_x = 2450
roi_y = 4730
roi_size = 200
pixel_size = 0.2125  # microns per pixel

print("="*80)
print("VITAMIN-P vs XENIUM: ROI VISUALIZATION WITH TRANSCRIPTS")
print("="*80)

# ============================================================================
# LOAD DATA
# ============================================================================
print("\n[1] Loading morphology image...")
with tifffile.TiffFile(morphology_image) as tif:
    img = tif.asarray()
print(f"  Image shape: {img.shape}")

# Extract ROI from both channels
if len(img.shape) == 3 and img.shape[0] == 2:
    dapi_roi = img[0, roi_y:roi_y+roi_size, roi_x:roi_x+roi_size]
    cell_roi = img[1, roi_y:roi_y+roi_size, roi_x:roi_x+roi_size]
else:
    dapi_roi = img[roi_y:roi_y+roi_size, roi_x:roi_x+roi_size]
    cell_roi = dapi_roi.copy()

print(f"  ROI extracted: ({roi_x}, {roi_y}) -> ({roi_x+roi_size}, {roi_y+roi_size})")

# ============================================================================
# LOAD TRANSCRIPTS
# ============================================================================
print("\n[2] Loading transcripts...")
transcripts = pd.read_parquet(f"{data_dir}/transcripts.parquet")

# Get crop info from xenium geojson
with open(xenium_geojson, 'r') as f:
    xenium_data = json.load(f)
crop_info = xenium_data['metadata']['crop_region']
x_start = crop_info['x_start']
y_start = crop_info['y_start']

# Convert transcripts to pixel coordinates
transcripts['x_pixels'] = transcripts['x_location'] / pixel_size
transcripts['y_pixels'] = transcripts['y_location'] / pixel_size

# Filter to ROI
transcripts_roi = transcripts[
    (transcripts['x_pixels'] >= x_start + roi_x) & 
    (transcripts['x_pixels'] < x_start + roi_x + roi_size) &
    (transcripts['y_pixels'] >= y_start + roi_y) & 
    (transcripts['y_pixels'] < y_start + roi_y + roi_size)
].copy()

# Convert to local ROI coordinates
transcripts_roi['x_local'] = transcripts_roi['x_pixels'] - (x_start + roi_x)
transcripts_roi['y_local'] = transcripts_roi['y_pixels'] - (y_start + roi_y)

print(f"  Total transcripts in ROI: {len(transcripts_roi):,}")

# Sample MORE transcripts for visualization (30% for better density)
sample_frac = 0.3
transcripts_sample = transcripts_roi.sample(frac=sample_frac, random_state=42)
print(f"  Sampled transcripts for display: {len(transcripts_sample):,}")

# ============================================================================
# LOAD SEGMENTATIONS
# ============================================================================
print("\n[3] Loading segmentations...")

with open(vitamin_p_geojson, 'r') as f:
    vitamin_p_data = json.load(f)
vitamin_p_features = vitamin_p_data['features']
print(f"  Vitamin-P cells loaded: {len(vitamin_p_features):,}")

with open(xenium_geojson, 'r') as f:
    xenium_data = json.load(f)
xenium_features = xenium_data['features']
print(f"  Xenium cells loaded: {len(xenium_features):,}")

# ============================================================================
# EXTRACT CELLS IN ROI
# ============================================================================
print("\n[4] Extracting cells in ROI...")

def extract_roi_cells(features, roi_x, roi_y, roi_size):
    """Extract cells that overlap with ROI"""
    roi_cells = []
    
    for feature in features:
        coords = feature['geometry']['coordinates']
        if isinstance(coords[0][0][0], list):
            vertices = coords[0][0]
        else:
            vertices = coords[0]
        
        vertices = np.array(vertices)
        
        x_in = (vertices[:, 0] >= roi_x) & (vertices[:, 0] < roi_x + roi_size)
        y_in = (vertices[:, 1] >= roi_y) & (vertices[:, 1] < roi_y + roi_size)
        
        if np.any(x_in & y_in):
            vertices_local = vertices - np.array([roi_x, roi_y])
            roi_cells.append(vertices_local)
    
    return roi_cells

vp_roi_cells = extract_roi_cells(vitamin_p_features, roi_x, roi_y, roi_size)
xen_roi_cells = extract_roi_cells(xenium_features, roi_x, roi_y, roi_size)

print(f"  Vitamin-P cells in ROI: {len(vp_roi_cells)}")
print(f"  Xenium cells in ROI: {len(xen_roi_cells)}")

# ============================================================================
# VISUALIZATION
# ============================================================================
print("\n[5] Creating visualization...")

# Set PDF backend for editable output
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42  # TrueType fonts (editable text)
matplotlib.rcParams['ps.fonttype'] = 42   # TrueType fonts

fig, axes = plt.subplots(1, 2, figsize=(20, 9))

# Normalize images with brightness enhancement
def normalize_bright(img, percentile_low=1, percentile_high=99.5, boost=2.0):
    p_low = np.percentile(img, percentile_low)
    p_high = np.percentile(img, percentile_high)
    img_norm = np.clip((img - p_low) / (p_high - p_low + 1e-8), 0, 1)
    img_bright = np.power(img_norm, 1/boost)
    return img_bright

# Apply different brightness to each channel
dapi_norm = normalize_bright(dapi_roi, boost=2.5)
cell_norm = normalize_bright(cell_roi, boost=2.0)  # Less boost = darker green

# Further dim the green channel
cell_norm = cell_norm * 0.6  # Reduce green intensity by 40%

# Create RGB composite
rgb_vp = np.zeros((roi_size, roi_size, 3))
rgb_vp[:, :, 2] = dapi_norm  # Blue
rgb_vp[:, :, 1] = cell_norm  # Green (darker)

rgb_xen = rgb_vp.copy()

# Get top genes by count and assign distinct colors
gene_counts = transcripts_sample['feature_name'].value_counts()
top_genes = gene_counts.head(20).index.tolist()

# Create a brighter, more vibrant color palette
colors_bright = [
    [1.0, 0.2, 0.2, 1.0],  # Bright red
    [1.0, 0.8, 0.0, 1.0],  # Bright yellow
    [0.0, 1.0, 0.5, 1.0],  # Bright cyan-green
    [1.0, 0.0, 1.0, 1.0],  # Bright magenta
    [0.0, 0.8, 1.0, 1.0],  # Bright cyan
    [1.0, 0.5, 0.0, 1.0],  # Bright orange
    [0.5, 1.0, 0.0, 1.0],  # Bright lime
    [1.0, 0.0, 0.5, 1.0],  # Bright pink
    [0.5, 0.0, 1.0, 1.0],  # Bright purple
    [1.0, 1.0, 0.0, 1.0],  # Bright yellow
    [0.0, 1.0, 1.0, 1.0],  # Bright aqua
    [1.0, 0.3, 0.7, 1.0],  # Hot pink
    [0.7, 1.0, 0.3, 1.0],  # Yellow-green
    [0.3, 0.7, 1.0, 1.0],  # Sky blue
    [1.0, 0.7, 0.3, 1.0],  # Light orange
    [0.9, 0.3, 0.9, 1.0],  # Light magenta
    [0.3, 0.9, 0.9, 1.0],  # Turquoise
    [0.9, 0.9, 0.3, 1.0],  # Olive yellow
    [0.3, 0.9, 0.3, 1.0],  # Light green
    [0.9, 0.3, 0.3, 1.0],  # Coral
]

gene_to_color = {gene: colors_bright[i] for i, gene in enumerate(top_genes)}

# Assign lighter gray to other genes
for gene in transcripts_sample['feature_name'].unique():
    if gene not in gene_to_color:
        gene_to_color[gene] = [0.8, 0.8, 0.8, 0.7]  # Light gray

# --- Vitamin-P panel ---
axes[0].imshow(rgb_vp, extent=[0, roi_size, roi_size, 0], aspect='equal')
axes[0].set_xlim(0, roi_size)
axes[0].set_ylim(roi_size, 0)

# Draw cell boundaries
for vertices in vp_roi_cells:
    poly = Polygon(vertices, fill=False, edgecolor='red', linewidth=1.5, alpha=0.8)
    axes[0].add_patch(poly)

# Draw transcripts by gene group - LARGER, NO EDGE
for gene in top_genes:
    gene_tx = transcripts_sample[transcripts_sample['feature_name'] == gene]
    if len(gene_tx) > 0:
        axes[0].scatter(gene_tx['x_local'], gene_tx['y_local'], 
                       c=[gene_to_color[gene]], s=12, alpha=1.0, 
                       edgecolors='none', linewidths=0, label=gene)

axes[0].set_title(f'Vitamin-P Segmentation\n{len(vp_roi_cells)} cells, {len(transcripts_sample):,} transcripts', 
                 fontsize=14, fontweight='bold')
axes[0].set_xlabel('X (pixels)', fontsize=12)
axes[0].set_ylabel('Y (pixels)', fontsize=12)
axes[0].grid(False)

# --- Xenium panel ---
axes[1].imshow(rgb_xen, extent=[0, roi_size, roi_size, 0], aspect='equal')
axes[1].set_xlim(0, roi_size)
axes[1].set_ylim(roi_size, 0)

# Draw cell boundaries
for vertices in xen_roi_cells:
    poly = Polygon(vertices, fill=False, edgecolor='red', linewidth=1.5, alpha=0.8)
    axes[1].add_patch(poly)

# Draw transcripts by gene group - LARGER, NO EDGE
for gene in top_genes:
    gene_tx = transcripts_sample[transcripts_sample['feature_name'] == gene]
    if len(gene_tx) > 0:
        axes[1].scatter(gene_tx['x_local'], gene_tx['y_local'], 
                       c=[gene_to_color[gene]], s=12, alpha=1.0, 
                       edgecolors='none', linewidths=0)

axes[1].set_title(f'Xenium Segmentation\n{len(xen_roi_cells)} cells, {len(transcripts_sample):,} transcripts', 
                 fontsize=14, fontweight='bold')
axes[1].set_xlabel('X (pixels)', fontsize=12)
axes[1].set_ylabel('Y (pixels)', fontsize=12)
axes[1].grid(False)

# Add legend
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], color='blue', lw=4, label='DAPI (nuclei)'),
    Line2D([0], [0], color='green', lw=4, label='Cell marker'),
    Line2D([0], [0], color='red', lw=2, label='Cell boundaries'),
]

# Add top 5 genes to legend with brighter colors
for i, gene in enumerate(top_genes[:5]):
    legend_elements.append(
        Line2D([0], [0], marker='o', color='w', 
               markerfacecolor=gene_to_color[gene], 
               markersize=10, label=f'{gene} ({gene_counts[gene]})', 
               linestyle='None', markeredgecolor='none', markeredgewidth=0)
    )

axes[0].legend(handles=legend_elements, loc='upper right', fontsize=9, framealpha=0.9)
axes[1].legend(handles=legend_elements, loc='upper right', fontsize=9, framealpha=0.9)

plt.tight_layout()

# Save as editable PDF
output_pdf = "vitamin_p_vs_xenium_comparison.pdf"
plt.savefig(output_pdf, format='pdf', dpi=300, bbox_inches='tight')
print(f"\n✅ Saved editable PDF: {output_pdf}")

# Also save individual panels
fig_vp, ax_vp = plt.subplots(1, 1, figsize=(10, 9))
ax_vp.imshow(rgb_vp, extent=[0, roi_size, roi_size, 0], aspect='equal')
ax_vp.set_xlim(0, roi_size)
ax_vp.set_ylim(roi_size, 0)
for vertices in vp_roi_cells:
    poly = Polygon(vertices, fill=False, edgecolor='red', linewidth=1.5, alpha=0.8)
    ax_vp.add_patch(poly)
for gene in top_genes:
    gene_tx = transcripts_sample[transcripts_sample['feature_name'] == gene]
    if len(gene_tx) > 0:
        ax_vp.scatter(gene_tx['x_local'], gene_tx['y_local'], 
                     c=[gene_to_color[gene]], s=12, alpha=1.0, 
                     edgecolors='none', linewidths=0)
ax_vp.set_title(f'Vitamin-P Segmentation\n{len(vp_roi_cells)} cells, {len(transcripts_sample):,} transcripts', 
               fontsize=14, fontweight='bold')
ax_vp.set_xlabel('X (pixels)', fontsize=12)
ax_vp.set_ylabel('Y (pixels)', fontsize=12)
ax_vp.legend(handles=legend_elements, loc='upper right', fontsize=9, framealpha=0.9)
ax_vp.grid(False)
plt.tight_layout()
plt.savefig("vitamin_p_panel.pdf", format='pdf', dpi=300, bbox_inches='tight')
print(f"✅ Saved: vitamin_p_panel.pdf")
plt.close()

fig_xen, ax_xen = plt.subplots(1, 1, figsize=(10, 9))
ax_xen.imshow(rgb_xen, extent=[0, roi_size, roi_size, 0], aspect='equal')
ax_xen.set_xlim(0, roi_size)
ax_xen.set_ylim(roi_size, 0)
for vertices in xen_roi_cells:
    poly = Polygon(vertices, fill=False, edgecolor='red', linewidth=1.5, alpha=0.8)
    ax_xen.add_patch(poly)
for gene in top_genes:
    gene_tx = transcripts_sample[transcripts_sample['feature_name'] == gene]
    if len(gene_tx) > 0:
        ax_xen.scatter(gene_tx['x_local'], gene_tx['y_local'], 
                      c=[gene_to_color[gene]], s=12, alpha=1.0, 
                      edgecolors='none', linewidths=0)
ax_xen.set_title(f'Xenium Segmentation\n{len(xen_roi_cells)} cells, {len(transcripts_sample):,} transcripts', 
                fontsize=14, fontweight='bold')
ax_xen.set_xlabel('X (pixels)', fontsize=12)
ax_xen.set_ylabel('Y (pixels)', fontsize=12)
ax_xen.legend(handles=legend_elements, loc='upper right', fontsize=9, framealpha=0.9)
ax_xen.grid(False)
plt.tight_layout()
plt.savefig("xenium_panel.pdf", format='pdf', dpi=300, bbox_inches='tight')
print(f"✅ Saved: xenium_panel.pdf")
plt.close()

plt.show()

print("\n✅ Visualization complete!")
print(f"  Showing {len(transcripts_sample):,} transcripts ({sample_frac*100:.0f}% sample)")
print(f"  Top genes displayed with distinct colors")
print("\n📄 Saved files:")
print("  • vitamin_p_vs_xenium_comparison.pdf (both panels)")
print("  • vitamin_p_panel.pdf (individual)")
print("  • xenium_panel.pdf (individual)")
print("\nThese PDFs are fully editable in Adobe Illustrator!")
print("="*80)

VITAMIN-P vs XENIUM: ROI VISUALIZATION WITH TRANSCRIPTS

[1] Loading morphology image...


FileNotFoundError: [Errno 2] No such file or directory: '/home/yshokrollahi/test_images/combined_2channel_crop_10000x10000.ome.tif'

In [3]:
!!pip install pyarrow

['Defaulting to user installation because normal site-packages is not writeable',
 'Requirement already satisfied: pyarrow in /home/yshokrollahi/.local/lib/python3.11/site-packages (23.0.0)',
 '',
 '\x1b[\x1b\x1bnotice\x1b\x1b[1;39;49m]\x1b\x1b A new release of pip is available: \x1b\x1b23.3.1\x1b\x1b -> \x1b\x1b25.3\x1b',
 '\x1b[\x1b\x1bnotice\x1b\x1b[1;39;49m]\x1b\x1b To update, run: \x1b\x1bpip install --upgrade pip\x1b']